#importing and getting the docker Image

In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import json
import numpy as np
import os
import time

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")


'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


#Getting the im2rec.py to convert our images to RecordIO files

In [2]:
%%bash
wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py
chmod +x im2rec.py

--2020-09-15 04:52:19--  https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15838 (15K) [text/plain]
Saving to: ‘im2rec.py’

     0K .......... .....                                      100% 13.1M=0.001s

2020-09-15 04:52:19 (13.1 MB/s) - ‘im2rec.py’ saved [15838/15838]



Creating .lst for our training and testing images

In [2]:
%%bash
python im2rec.py --list --recursive  --test-ratio 0.3 --train-ratio 0.7 imag_2 DATASet-Collected-by-MushroomLess/

Agaricus bisporus 0
Agrocybe praecox 1
Amethyst deceiver 2
Bay Boltete 3
Beefasteak fungus 4
Birch bolete 5
Bitter beech bolete 6
Bitter bolete 7
Black truffle 8
Blackening brittlegill 9
Bloody Milk cap 10
Blusher 11
Blushing bracket 12
Bovine bolete 13
Brick cap 14
Brown roll-rim 15
Buttery Collybia 16
Caesar's Mushroom 17
Cep 18
Chanterelle 19
Charcoal burner 20
Chicken of the Wood 21
Clouded agaric 22
Collared earthstar 23
Common Puffbal 24
Common Stinkhorn 25
Coprinellus micaceus 26
Coprinus comatus 27
Dark Cep 28
Dead man's fingers 29
Deadly dapperling 30
Deadly webcap 31
Death cap 32
Deceiving bolete 33
Destroying angel 34
Earthy inocybe 35
False champignon 36
False chanterelle 37
False saffron milkcap 38
False turkey tail 39
Field Mushroom 40
Fly agaric 41
Fool's mushroom 42
Fools webcap 43
Fried chicken mushroom 44
Funnel Chanterelle 45
Galerina marginata 46
Gassy webcap 47
Gemmed Amanita 48
Goblet funnel cap 49
Gypsy Mushroom 50
Gyromitra esculent 51
Hedgehog 52
Hen of the Woo

#Generating the .Rec Files and meanwhile resizing them to "224" 

In [3]:
%%bash
python im2rec.py --num-thread=4  --resize 421 imag_2_train.lst DATASet-Collected-by-MushroomLess/
python im2rec.py --num-thread=4  --resize 421 imag_2_test.lst DATASet-Collected-by-MushroomLess/

Creating .rec file from /home/ec2-user/SageMaker/MushroomLess-Project/imag_2_train.lst in /home/ec2-user/SageMaker/MushroomLess-Project
time: 0.11029839515686035  count: 0
time: 55.5292706489563  count: 1000
time: 45.31613278388977  count: 2000
time: 35.713550329208374  count: 3000
time: 30.96580457687378  count: 4000
Creating .rec file from /home/ec2-user/SageMaker/MushroomLess-Project/imag_2_test.lst in /home/ec2-user/SageMaker/MushroomLess-Project
time: 0.016009092330932617  count: 0
time: 43.1432421207428  count: 1000
time: 37.68852925300598  count: 2000


# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using

In [4]:
%%bash
aws s3 cp imag_2_train.rec s3://data-text-or-image-2020-9-15-7-41/imagedata/input/recordio/train2/
aws s3 cp imag_2_test.rec  s3://data-text-or-image-2020-9-15-7-41/imagedata/input/recordio/test2/

upload: ./imag_2_train.rec to s3://data-text-or-image-2020-9-15-7-41/imagedata/input/recordio/train2/imag_2_train.rec
upload: ./imag_2_test.rec to s3://data-text-or-image-2020-9-15-7-41/imagedata/input/recordio/test2/imag_2_test.rec


In [2]:
s3train_path = 's3://data-text-or-image-2020-9-15-7-41/imagedata/input/recordio/train2/'
s3validation_path = 's3://data-text-or-image-2020-9-15-7-41/imagedata/input/recordio/test2/'

#Giving the S3Data_path for our ML - Channels

In [3]:
train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


#Specifying the Output path for model.
and giving the Image-Docker
the needed configuration 

In [4]:
output_path='s3://data-text-or-image-2020-9-15-7-41/imagedata/output2/'
image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p2.xlarge',
    output_path=output_path,
    train_volume_size=50,
    sagemaker_session=sess
)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [5]:
num_classes=127
num_training=4781
mini_size_batch=10
num_layers=18
image_shape="3,421,421"
epochs=40
learning_rate=.01
use_pretrained=1

#Specifying the hyperparameters for Our Model.

In [6]:
base_hyperparameters=dict(
    use_pretrained_model=use_pretrained,
    image_shape=image_shape,
    num_classes=num_classes,
    num_training_samples=num_training,
)
hyperparameters={
    **base_hyperparameters, 
    **dict(
        learning_rate=learning_rate,
        mini_batch_size=mini_size_batch,
        optimizer='sgd',
        early_stopping=True,
        early_stopping_min_epochs=5
    )
}
image_classifier.set_hyperparameters(**hyperparameters)
hyperparameters

{'use_pretrained_model': 1,
 'image_shape': '3,421,421',
 'num_classes': 127,
 'num_training_samples': 4781,
 'learning_rate': 0.01,
 'mini_batch_size': 10,
 'optimizer': 'sgd',
 'early_stopping': True,
 'early_stopping_min_epochs': 5}

#Let the training start!

In [ ]:
%%time

now = str(int(time.time()))
training_job_name = 'IC-' +'MushroomLess-'+ now
image_classifier.fit(inputs=data_channels, job_name=training_job_name, logs=True)
job = image_classifier.latest_training_job

2020-09-16 11:06:49 Starting - Starting the training job...
2020-09-16 11:06:51 Starting - Launching requested ML instances.........
2020-09-16 11:08:23 Starting - Preparing the instances for training............
2020-09-16 11:10:23 Downloading - Downloading input data...
2020-09-16 11:11:02 Training - Downloading the training image...
2020-09-16 11:11:47 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
[09/16/2020 11:11:51 INFO 140519327504192] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[09/16/2020 11:11:51 INFO 14051932750

#Deploying the model to the Cloud

In [9]:
%%time
deployed_endpoint = image_classifier.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.medium'
)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: IC-MushroomLess-1600233742


---------------------!CPU times: user 381 ms, sys: 474 µs, total: 382 ms
Wall time: 10min 34s


In [10]:
deployed_endpoint.endpoint

'IC-MushroomLess-1600233742'

#Testing The Model

In [15]:
import json
import numpy as np
import os
image_x="vx.jpg"
def classify_deployed(image_file, classes):
    payload = None
    with open(image_file, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)
        
    deployed_endpoint.content_type = 'application/x-image'
    result = json.loads(deployed_endpoint.predict(payload))
    best_prob_index = np.argmax(result)
    return (classes[best_prob_index], result[best_prob_index])

classes=['Agaricus bisporus',
'Agrocybe praecox',
'Amethyst deceiver',
'Bay Boltete',
'Beefasteak fungus',
'Birch bolete',
'Bitter beech bolete',
'Bitter bolete',
'Black truffle',
'Blackening brittlegill',
'Bloody Milk cap',
'Blusher',
'Blushing bracket',
'Bovine bolete',
'Brick cap',
'Brown roll-rim',
'Buttery Collybia',
"Caesar's Mushroom",
'Cep',
'Chanterelle',
'Charcoal burner',
'Chicken of the Wood',
'Clouded agaric',
'Collared earthstar',
'Common Puffbal',
'Common Stinkhorn',
'Coprinellus micaceus',
'Coprinus comatus',
'Dark Cep',
"Dead man's fingers",
'Deadly dapperling',
'Deadly webcap',
'Death cap',
'Deceiving bolete',
'Destroying angel',
'Earthy inocybe',
'False champignon',
'False chanterelle',
'False saffron milkcap',
'False turkey tail',
'Field Mushroom',
'Fly agaric',
"Fool's mushroom",
'Fools webcap',
'Fried chicken mushroom',
'Funnel Chanterelle',
'Galerina marginata',
'Gassy webcap',
'Gemmed Amanita',
'Goblet funnel cap',
'Gypsy Mushroom',
'Gyromitra esculent',
'Hedgehog',
'Hen of the Woods',
'Honey fungus',
'Horn of plenty',
'Horse Musroom',
'Inky cap',
'Inocybe Rimosa',
'Inocybe erubescens',
'King trumpet Mushroom',
'Lacrymaria lacrymabunda',
'Latticed stinkhorn',
'Leccinum aurantiacum',
'Lilac bonnet',
'Lingzhi mushroom',
'Livid entoloma',
'Lurid bolete',
'Lyophyllum connatum',
'Macrolepiota Procera',
'Megacollybia platyphylla',
'Miller',
'Morchella',
"Mower's Mushroom",
'Neoboletus luridiformis',
'Omphalotus olearius',
'Oyster',
'Panther cap',
'Peppery Milkcap',
'Peppery bolete',
'Pine Bolete',
'Pine cone mushroom',
'Plums and Custard',
'Poison Pie',
'Psathyrella candolleana',
'Psilocybe cubensis',
'Rooting bolete',
'Rosy bonnet',
'Royal fly agaric',
'Russula Ochroleuca',
'Russula integra',
'Russula vesca',
'Saffron milk cap',
'Sarcosphaera coronaria',
"Satan's bolete",
'Scaly Wood Mushroom',
'Scarlet cup',
'Scarlet hood',
'Scotch Bonnet',
'Smoky polypore',
'Spiny puffball',
'Spotted tricholoma',
"St. George's Mushroom",
'Stinking russula',
'Stubble rosegill',
'Suillus brevipes',
'Suillus granulatus',
'Suillus luteus',
'Sulphur tuft',
'Summer cep',
'The prince',
'The sickener',
'Tinder fungus',
'Torn fibrecap',
'Trametes versicolor',
'Tricholoma scalpturatum',
'Velvet-top fungus',
'Violet webcap',
'Warted Amanita',
'White Saddle',
"Witch's hat",
'Wood blewit',
'Woolly  milkcap',
'Xerocomellus chrysenteron',
'Yellow knight',
'Yellow-stainers',
'shiitake']
print(classify_deployed(image_x,classes))

('Psilocybe cubensis', 0.7514241337776184)
